In [ ]:
!pip install ibm-watsonx-ai
!pip install sentence_transformers
!pip install PyPDF2
!pip install docling
!pip install faiss-cpu
!pip install ibm_cloud_sdk_core
!pip install ibm-watson

In [53]:
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference
from dotenv import load_dotenv
import os
from bs4 import BeautifulSoup
import markdown
import os
from docling.document_converter import DocumentConverter
import pandas as pd
import json
import openai
import time
from PyPDF2 import PdfReader

load_dotenv()

ENDPOINT_URL =      os.getenv('ENDPOINT_URL')
API_KEY =           os.getenv('API_KEY')
PROJECT_ID =        os.getenv('PROJECT_ID')
openai.api_key =    os.getenv('OPENAI_API_KEY')

def chat(query, max_tokens=8000):
    credentials = Credentials(
        url=ENDPOINT_URL,
        api_key=API_KEY
    )

    model = ModelInference(
        model_id="meta-llama/llama-3-3-70b-instruct",
        credentials=credentials,
        project_id=PROJECT_ID,
        params={
            "max_tokens": max_tokens,
            "temperature": 0.0
        }
    )

    result = model.chat(messages=[{'role': 'user', 'content': query}])

    answer = result['choices'][0]['message']['content']

    return answer

def gpt_chat(prompts: list,
         model: str = "gpt-4o-mini",
         temperature: float = 0.6,
         max_tokens: int = 8000,
         frequency_penalty: float = 0.0,
         presence_penalty: float = 0.0,
         format: str = "text",
         clean_output: bool = True,
         verbose: bool = True) -> tuple:

    if not isinstance(prompts[0], dict):
        messages = []
        len_prompt = len(prompts)

        if len_prompt == 1:
            messages.append({"role": "system",  "content": "Sei un ottimo assistente."})
            messages.append({"role": "user",    "content": prompts[0]})
        
        else:
            for i in range(len_prompt):
                if i == 0:
                    messages.append({"role": "system",      "content": prompts[i]})
                elif i%2 != 0:
                    messages.append({"role": "user",        "content": prompts[i]})
                else:
                    messages.append({"role": "assistant",   "content": prompts[i]})
    else:
        messages = prompts
    
    # Setting output to JSON or STRING
    if format == "json":
        if model == "gpt-4o":
            model = "gpt-4o"

        elif model == "gpt-4":
            model = "gpt-4-turbo"

        else:
            model = "gpt-4o-mini"

        output_format = "json_object"

    elif format == "text":
        model = model
        output_format = "text"
    
    else:
        raise Exception(f"The format `{format}` is not compatible. Choose between `text` or `json`.")


    # Iterating chat N times
    retries = 0
    max_retries = 3
    while retries < max_retries:
        try:
            print(f'\n\tAsking {model} using a temperature of {temperature}...\n')
            completion = openai.chat.completions.create(
                model=model,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
                timeout=300,
                #functions=functions,
                frequency_penalty=frequency_penalty,
                presence_penalty=presence_penalty,
                response_format={"type": output_format}
            )

            answer = completion.choices[0].message.content

            ## Cleaning output
            if clean_output:
                answer = answer.replace('\n\n','\n')
                answer = answer.replace('\n',' ') 

                if format == "json":
                    try:
                        answer = json.loads(answer)
                    except:
                        answer = answer
       
                else:
                    ## Removing double or single quotes
                    answer = answer.strip("\"")
                    answer = answer.strip("\'")

            if verbose:
                print(answer)

            return answer, completion

        except openai.APIError as e:
            print(e)
            print('Timeout error, retrying...')
            retries += 1
            time.sleep(5 * retries)

    return print("Server is not responding.")

## Retrieving

In [56]:
def extract_text_from_pdf_OLD(pdf_path):
    converter = DocumentConverter()
    result = converter.convert(pdf_path)
    output = result.document.export_to_markdown()
    return output

def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""

    # Itera sulle pagine e estrai il testo
    for page_num, page in enumerate(reader.pages, start=1):
        text += page.extract_text()
        text += "\n"
    
    return text

def extract_text_from_html(html_path):
    with open(html_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')
        output = soup.get_text()
        return output

def extract_text_from_md(md_path):
    with open(md_path, 'r', encoding='utf-8') as file:
        html = markdown.markdown(file.read())
        output = BeautifulSoup(html, 'html.parser').get_text()
        return output
    
def extract_text_from_docx(docx_path):
    converter = DocumentConverter()
    result = converter.convert(docx_path)
    output = result.document.export_to_text()
    return output

def extract_text_from_csv(csv_path):
    df = pd.read_csv(csv_path)
    output = df.to_string()
    return output

def save_text(text, output_path):
    with open(output_path, 'w', encoding='utf-8') as file:
        file.write(text)

# Directory di input/output
folders = ['blogpost', 'codice_galattico', 'menu', 'misc']

for folder in folders:
    input_dir = f"dataset/{folder}"
    output_dir = f"converted_dataset/{folder}"

    for file_name in os.listdir(input_dir):
        file_path =     os.path.join(input_dir, file_name)
        output_path =   os.path.join(output_dir, f"{os.path.splitext(file_name)[0]}.txt")

        if not os.path.isfile(output_path):
            if file_name.endswith('.pdf'):
                text = extract_text_from_pdf(file_path)
            elif file_name.endswith('.html'):
                text = extract_text_from_html(file_path)
            elif file_name.endswith('.md'):
                text = extract_text_from_md(file_path)
            elif file_name.endswith('.docx'):
                text = extract_text_from_docx(file_path)
            elif file_name.endswith('.csv'):
                text = extract_text_from_csv(file_path)
            else:
                continue
        else:
            print("File already extracted.")
            continue

        save_text(text, output_path)

File already extracted.
File already extracted.
File already extracted.
File already extracted.
File already extracted.
File already extracted.


In [55]:
path = 'dataset_md/Eco di Pandora.txt'
path = 'dataset_md/L Eco dei Sapori.txt'
path = 'dataset_md/L Essenza delle Dune.txt'
orders_path = 'converted_dataset/codice_galattico/Codice Galattico.txt'

with open(path, 'r') as f:
    text = f.read()

with open(orders_path, 'r') as f:
    text_orders = f.read()

splitted_text = text.split('## Menu')

## First section
request = f"""Segui attentamente queste istruzioni. Devi estrarre le seguenti informazioni dal <DOCUMENTO> che ti fornirò.
Crea un file JSON, ovvero una lista di dizionari, uno per ogni ricetta, composto da 3 chiavi:
- *chef*: stringa
- *pianeta*: stringa
- *ristorante*: stringa
- *skills*: lista. Il primo elemento è una skill/competenza, il secondo elemento è un numero intero. A volte un numero può essere rappresentato da un numero romano o altro formato. Convertilo sempre in numero arabo intero. 
Se un valore non è presente, scrivi "none" come valore.

<DOCUMENTO>: "{splitted_text[0]}"
"""

output_0 = chat(request)
output_0 = output_0.split('```')[1]
output_0 = output_0.replace('json', '')
output_0 = json.loads(output_0)
print(output_0)

## Second section
request = f"""Segui attentamente queste istruzioni. Devi estrarre le seguenti informazioni dal <DOCUMENTO> che ti fornirò.
Crea un file JSON, ovvero una lista di dizionari, uno per ogni ricetta, composto da 5 chiavi:
- *nome_ricetta*
- *ingredienti*
- *tecniche*
- *descrizione*
- *ordine_riferimento*: questo dato non è un numero, bensì un valore che fa riferimento a ordini professionali. Trovi degli esempi all'interno del documento <DOCUMENTO ORDINI>. Spesso questo valore non è presente o è indicato da un'immagine.
Se un valore non è presente, scrivi "none" come valore.

<DOCUMENTO>: "{splitted_text[1]};
<DOCUMENTO ORDINI> "{text_orders}"
"""

output_1 = chat(request)
if type(output_1).__name__ == 'str':
    output_1 = output_1.split('```')[1]
    output_1 = output_1.replace('json', '')
    output_1 = json.loads(output_1)
print(output_1)

KeyboardInterrupt: 

In [46]:
output_1

[{'nome_ricetta': 'Il Ricordo del Fuoco Celeste',
  'ingredienti': ['Farina di Nettuno',
   'Carne di Drago',
   'Lattuga Namecciana',
   'Salsa Szechuan',
   'Shard di Materia Oscura',
   'Uova di Fenice',
   'Spaghi del Sole'],
  'tecniche': ['Amalgamazione Sintetica Molecolare',
   'Ebollizione Magneto-Cinetica Pulsante',
   'Cryo-Tessitura Energetica Polarizzata'],
  'descrizione': "Un piatto che combina sapori e tecniche avanzate per creare un'esperienza culinaria unica.",
  'ordine_riferimento': 'none'},
 {'nome_ricetta': 'Sinfonia dei Ricordi Celesti',
  'ingredienti': ['Carne di balena spaziale',
   'Granuli di nebbia arcobaleno',
   'Impasto gravitazionale vorticoso'],
  'tecniche': ['Spore quantiche',
   'Vapore termocinetico multiplo',
   'Sinergia Elettro-Osmotica Programmabile'],
  'descrizione': 'Un viaggio culinario che esplora i sapori come frammenti del passato.',
  'ordine_riferimento': 'none'},
 {'nome_ricetta': 'Sinfonia del Cosmo',
  'ingredienti': ['Liane di Plasm

In [50]:
output_1[3]

{'nome_ricetta': 'Rapsodia dei Ricordi Celesti',
 'ingredienti': ['Carne di Xenodonte',
  'Carne di Mucca',
  'Forno Dinamico Inversionale',
  'Lacrime di Andromeda',
  'Essenza di Speziaria'],
 'tecniche': ['Marinatura Sotto Zero a Polarità Inversa',
  'Cryo-Tessitura Energetica Polarizzata'],
 'descrizione': 'Un viaggio gastronomico che combina sapori e memorie.',
 'ordine_riferimento': 'none'}

## Data Store (Chroma)

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_chroma import Chroma

filename = "converted_dataset/Anima Cosmica.txt"

loader = TextLoader(filename)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

##  Embeddings

In [ ]:
from langchain_ibm import WatsonxEmbeddings
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes

parameters = {
    "decoding_method": "sample",
    "max_new_tokens": 100,
    "min_new_tokens": 1,
    "temperature": 0.5,
    "top_k": 50,
    "top_p": 1,
}

embeddings = WatsonxEmbeddings(
    model_id="ibm/slate-30m-english-rtrvr",
    url=ENDPOINT_URL
    apikey=API_KEY
    project_id=PROJECT_ID,
    params = parameters
    )

docsearch = Chroma.from_documents(texts, embeddings)

from sentence_transformers import SentenceTransformer
import faiss
import os

# Modello per gli embedding
model = SentenceTransformer('all-MiniLM-L6-v2')

# Path dei documenti
input_dir = "converted_dataset/"
texts = []

# Legge i file e li suddivide in chunk
for file_name in os.listdir(input_dir):
    with open(os.path.join(input_dir, file_name), 'r', encoding='utf-8') as file:
        texts.append(file.read())

# Generazione degli embedding
embeddings = model.encode(texts)

# Creazione dell'indice FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Salvataggio dell'indice
faiss.write_index(index, "indice_faiss.index")

In [ ]:
from dotenv import load_dotenv
import openai
import os

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

# Import document
pdf_loader = PyPDFLoader('data_pdfreader/bando.pdf')
documents = pdf_loader.load()

# Split document
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(documents)

# Populate vector store
vectordb = Chroma.from_documents(
 documents=splits,
 embedding=OpenAIEmbeddings(),
 persist_directory='./data/memory'
)

vectordb.persist()

## Request
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-4", temperature=0.2),
    retriever=vectordb.as_retriever(),
    return_source_documents=True
)

In [ ]:
question = "Riassumi il contenuto del documento.\"\"\""
result = qa_chain.invoke({'query': question})

print(result['result'])

In [ ]:
## I RISULTATI NON SONO BUONI, MEGLIO CREARE UN DB PERSONALE (Maggiore Valore) ##

from libraries.bandit.bandit.functions import *

prompt = f"What is the average round for a single Pre-Seed Deeptech startup in Italy?"

rnd, _ = ask_langchain(prompt)

request = f"""
1. Estrai solo il <numero> dal seguente <testo>: "{rnd}".
2. Il numero deve essere in formato intero: ad esempio, se nel <testo> trovi scritto "€0,4 milioni", tu scrivi "400000". Non usare simboli di valuta. Formattalo come numero intero;
3. Crea un file JSON, ovvero un dizionario composto da due chiavi: "key" e "value";
3.1. "key" sarà sempre uguale a "round", mentre "value" sarà il <numero> che è stato estratto in precedenza."""

rnd_num, _ = chat([request], temperature = 0.0, format="json")
rnd_num

In [ ]:
from libraries.bandit.bandit.functions import *
from pypdf import PdfReader
import textwrap

# Set the width at which to wrap the text
width = 100

# creating a pdf reader object 
reader = PdfReader('data_pdfreader/bando.pdf') 
  
# creating a page object 
text = ""
for p in range(len(reader.pages)):
    pageObj = reader.pages[p]
    text += pageObj.extract_text()

In [ ]:
prompt = ["", f"BANDO: \"\"\"{text}\"\"\". Estrai solamente le macrosezioni *numerate* del documento senza nessun'altra informazione. Non scrivere le sottosezioni né nessun altra informazione non pertinente al nome della sezione. Forniscimi un elenco numerato dei vari punti. Non voglio nessuna frase introduttiva, voglio solo un elenco numerato."]

ans, _ = chat(prompt, model="gpt-4-turbo", temperature=0.0, max_tokens=4096, frequency_penalty=0.0, presence_penalty=0.0)
wrapped_text = textwrap.fill(ans, width=width)
print(wrapped_text)

In [ ]:
prompt = ["", f"BANDO: \"\"\"{text}\"\"\". Estrai solamente le macrosezioni *numerate* del documento senza nessun'altra informazione. Non scrivere le sottosezioni né nessun altra informazione non pertinente al nome della sezione. Forniscimi un elenco numerato dei vari punti. Non voglio nessuna frase introduttiva, voglio solo un elenco numerato."]

ans, _ = chat(prompt, model="gpt-4-turbo", temperature=0.0, max_tokens=4096, frequency_penalty=0.0, presence_penalty=0.0)
wrapped_text = textwrap.fill(ans, width=width)
print(wrapped_text)

## NEW

In [ ]:
!pip install crewai
!pip install langchain_openai
!pip install langchain_community

In [ ]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os
from crewai_tools import PDFSearchTool
from crewai_tools  import tool
from crewai import Crew
from crewai import Task
from crewai import Agent
from crewai.tools import BaseTool
from pydantic import Field
from langchain_community.utilities import GoogleSerperAPIWrapper

load_dotenv()

OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
SERPER_API_KEY=os.getenv("SERPER_API_KEY")

In [ ]:
llm=ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

Router_Agent = Agent(
  role='Router',
  goal='Route user question to a vectorstore or web search',
  backstory=(
    "You are an expert at routing a user question to a vectorstore or web search ."
    "Use the vectorstore for questions on transformer or differential transformer."
    "use web-search for question on latest news or recent topics."
    "use generation for generic questions otherwise"
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)
Retriever_Agent = Agent(
role="Retriever",
goal="Use the information retrieved from the vectorstore to answer the question",
backstory=(
    "You are an assistant for question-answering tasks."
    "Use the information present in the retrieved context to answer the question."
    "You have to provide a clear concise answer."
),
verbose=True,
allow_delegation=False,
llm=llm,
)

In [ ]:
search = GoogleSerperAPIWrapper

class SearchTool(BaseTool):
    name: str = "Search"
    description: str = "Useful for search-based queries. Use this to find current information about markets, companies, and trends."
    search: GoogleSerperAPIWrapper = Field(default_factory=GoogleSerperAPIWrapper)

    def _run(self, query: str) -> str:
        """Execute the search query and return results"""
        try:
            return self.search.run(query)
        except Exception as e:
            return f"Error performing search: {str(e)}"
class GenerationTool(BaseTool):
    name: str = "Generation_tool"
    description: str = "Useful for generic-based queries. Use this to find  information based on your own knowledge."
    #llm: ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

    def _run(self, query: str) -> str:
      llm=ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
      """Execute the search query and return results"""
      return llm.invoke(query)
generation_tool=GenerationTool()
web_search_tool = SearchTool()